In [2]:
from langchain_tavily import TavilySearch
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [4]:
web_search = TavilySearch(
    tavily_api_key=os.getenv("TAVILY_SEARCH_API_KEY"),
    max_results=5,
)

In [5]:
web_search_results  = web_search.invoke("Who is the Mayor of Kampala?")

In [6]:
print(web_search_results["results"][0]["content"])

The current Mayor of Kampala, Erias Lukwago, is a founding mayor of the MMC and has served on the Leadership Board since 2018. Kampala signed the Marrakech


In [10]:
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq

In [13]:
research_agent = create_react_agent(
    model= ChatGroq(
        model="llama3-8b-8192",
        groq_api_key=os.getenv("GROQ_API_KEY"),
    ),
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ), 
    name="Research Agent",
)

In [14]:
from langchain_core.messages import convert_to_messages

In [15]:
def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return
    
    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)

In [16]:
def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(".")[0]
        print(f"Update from subgraph: {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node: {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [17]:
for chunk in research_agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "Who is the Mayor of Kampala?",
            }
        ]
    }
):
    pretty_print_messages(chunk)

Update from node: agent:


================================== Ai Message ==================================
Name: Research Agent
Tool Calls:
  tavily_search (07736y6ah)
 Call ID: 07736y6ah
  Args:
    query: Who is the Mayor of Kampala?
    search_depth: advanced
    topic: general


Update from node: tools:


================================= Tool Message =================================
Name: tavily_search

{"query": "Who is the Mayor of Kampala?", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/Kampala_Capital_City_Authority", "title": "Kampala Capital City Authority - Wikipedia", "content": "| Office | Name of official | Title of officer |\n| --- | --- | --- |\n| Mayor | Kasirye Nganda Ali | Mayor of Makindye Division |\n| Mayor | Salim Uhuru | Mayor of Kampala Central Division |\n| Mayor | Paul Mugambe | Mayor of Nakawa Division |\n| Mayor | Emmanuel Serunjoji | Mayor of Kawempe Division |\n| Mayor | Mberaze Mawula Zac